# Week 8: Mission & Behavior Planning

## Module III: Planning & Decision Making

### Topics Covered

- Route Planning (Dijkstra, A* search on road network graphs)
- Heuristic Functions for Efficient Search
- Finite State Machines (FSM) for Behavior Decision Making
- Behavior Trees for Complex Scenarios
- Lane Change Decision Logic
- Intersection Handling

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Implement Dijkstra's algorithm and A* search for route planning on road networks
2. Design effective heuristic functions for A* search
3. Build Finite State Machines (FSMs) for driving behavior decisions
4. Understand behavior trees as an alternative to FSMs
5. Implement lane change decision logic with safety checks
6. Handle intersection scenarios with rule-based planning
7. Compare different planning approaches and understand trade-offs

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, defaultdict
import heapq
from enum import Enum
from typing import List, Tuple, Dict, Set

# Set random seed for reproducibility
np.random.seed(42)

## 1. Route Planning on Road Networks

Route planning determines the **high-level path** from origin to destination on a road network. This is the "mission" level - deciding which roads to take.

### Road Network as a Graph

**Nodes (Vertices):** Intersections, decision points
**Edges:** Road segments connecting nodes
**Weights:** Travel time, distance, or cost

### Graph Representation

$$G = (V, E)$$

Where:
- $V$: Set of vertices (nodes)
- $E$: Set of edges connecting vertices
- $w(u, v)$: Weight/cost of edge from $u$ to $v$

### Problem Formulation

**Input:**
- Graph $G = (V, E)$ with edge weights
- Start node $s$
- Goal node $g$

**Output:**
- Path $P = [s, v_1, v_2, \ldots, g]$ that minimizes total cost

**Cost Function:**
$$\text{cost}(P) = \sum_{i=0}^{n-1} w(v_i, v_{i+1})$$

### Applications in Autonomous Driving

1. **Navigation**: Route from current location to destination
2. **Rerouting**: Dynamic replanning when roads are blocked
3. **Multi-objective**: Balance time, distance, fuel, safety
4. **Time-dependent**: Account for traffic predictions

In [ ]:
# Complete Implementation of Dijkstra and A* on Road Network

class RoadNetwork:
    """Road network represented as a weighted graph."""
    
    def __init__(self):
        # Adjacency list: node -> list of (neighbor, weight)
        self.graph = defaultdict(list)
        # Node positions for visualization and heuristics
        self.positions = {}
        
    def add_edge(self, u, v, weight):
        """Add bidirectional edge to graph."""
        self.graph[u].append((v, weight))
        self.graph[v].append((u, weight))
    
    def add_node_position(self, node, x, y):
        """Store node's 2D position."""
        self.positions[node] = (x, y)
    
    def euclidean_heuristic(self, node, goal):
        """Euclidean distance heuristic for A*."""
        if node not in self.positions or goal not in self.positions:
            return 0
        x1, y1 = self.positions[node]
        x2, y2 = self.positions[goal]
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    def manhattan_heuristic(self, node, goal):
        """Manhattan distance heuristic."""
        if node not in self.positions or goal not in self.positions:
            return 0
        x1, y1 = self.positions[node]
        x2, y2 = self.positions[goal]
        return abs(x2 - x1) + abs(y2 - y1)


def dijkstra(network, start, goal):
    """
    Dijkstra's algorithm for shortest path.
    
    Returns:
        path: List of nodes from start to goal
        cost: Total cost of path
        explored: Set of explored nodes
    """
    # Initialize distances
    distances = {node: float('inf') for node in network.graph}
    distances[start] = 0
    
    # Track predecessors for path reconstruction
    predecessors = {}
    
    # Priority queue: (distance, node)
    pq = [(0, start)]
    
    # Track explored nodes
    explored = set()
    
    while pq:
        current_dist, current = heapq.heappop(pq)
        
        # Skip if already explored
        if current in explored:
            continue
        
        explored.add(current)
        
        # Found goal
        if current == goal:
            break
        
        # Explore neighbors
        for neighbor, weight in network.graph[current]:
            if neighbor in explored:
                continue
            
            new_dist = current_dist + weight
            
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                predecessors[neighbor] = current
                heapq.heappush(pq, (new_dist, neighbor))
    
    # Reconstruct path
    if goal not in predecessors and goal != start:
        return None, float('inf'), explored
    
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = predecessors[current]
    path.append(start)
    path.reverse()
    
    return path, distances[goal], explored


def astar(network, start, goal, heuristic_fn=None, weight=1.0):
    """
    A* search for shortest path.
    
    Args:
        network: RoadNetwork object
        start: Start node
        goal: Goal node
        heuristic_fn: Heuristic function (if None, uses Euclidean)
        weight: Heuristic weight (1.0 = optimal, >1.0 = weighted A*)
    
    Returns:
        path: List of nodes from start to goal
        cost: Total cost of path
        explored: Set of explored nodes
    """
    if heuristic_fn is None:
        heuristic_fn = network.euclidean_heuristic
    
    # g(n): cost from start to n
    g_score = {node: float('inf') for node in network.graph}
    g_score[start] = 0
    
    # f(n) = g(n) + h(n)
    f_score = {node: float('inf') for node in network.graph}
    f_score[start] = weight * heuristic_fn(start, goal)
    
    # Track predecessors
    predecessors = {}
    
    # Priority queue: (f_score, node)
    pq = [(f_score[start], start)]
    
    # Track explored nodes
    explored = set()
    
    while pq:
        current_f, current = heapq.heappop(pq)
        
        # Skip if already explored
        if current in explored:
            continue
        
        explored.add(current)
        
        # Found goal
        if current == goal:
            break
        
        # Explore neighbors
        for neighbor, weight_edge in network.graph[current]:
            if neighbor in explored:
                continue
            
            tentative_g = g_score[current] + weight_edge
            
            if tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + weight * heuristic_fn(neighbor, goal)
                predecessors[neighbor] = current
                heapq.heappush(pq, (f_score[neighbor], neighbor))
    
    # Reconstruct path
    if goal not in predecessors and goal != start:
        return None, float('inf'), explored
    
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = predecessors[current]
    path.append(start)
    path.reverse()
    
    return path, g_score[goal], explored


# Create a sample road network
def create_sample_road_network():
    """Create a sample city road network."""
    network = RoadNetwork()
    
    # Define nodes (intersections) with positions
    nodes = {
        'A': (0, 0),
        'B': (3, 1),
        'C': (1, 3),
        'D': (4, 4),
        'E': (7, 2),
        'F': (5, 5),
        'G': (8, 5),
        'H': (6, 7),
        'I': (9, 8),
        'J': (4, 8),
    }
    
    for node, pos in nodes.items():
        network.add_node_position(node, pos[0], pos[1])
    
    # Define edges (roads) with weights (travel time in minutes)
    edges = [
        ('A', 'B', 4),
        ('A', 'C', 3),
        ('B', 'C', 2),
        ('B', 'D', 5),
        ('B', 'E', 6),
        ('C', 'D', 3),
        ('C', 'J', 7),
        ('D', 'F', 2),
        ('D', 'E', 4),
        ('E', 'G', 3),
        ('F', 'H', 4),
        ('F', 'G', 4),
        ('G', 'I', 5),
        ('H', 'I', 3),
        ('H', 'J', 5),
    ]
    
    for u, v, w in edges:
        network.add_edge(u, v, w)
    
    return network


# Run comparison
network = create_sample_road_network()
start, goal = 'A', 'I'

# Run Dijkstra
dijkstra_path, dijkstra_cost, dijkstra_explored = dijkstra(network, start, goal)

# Run A* with Euclidean heuristic
astar_path, astar_cost, astar_explored = astar(network, start, goal)

# Run A* with Manhattan heuristic
astar_manhattan_path, astar_manhattan_cost, astar_manhattan_explored = astar(
    network, start, goal, heuristic_fn=network.manhattan_heuristic
)

# Run Weighted A* (ε = 1.5)
weighted_astar_path, weighted_astar_cost, weighted_astar_explored = astar(
    network, start, goal, weight=1.5
)

# Visualization
fig = plt.figure(figsize=(18, 12))

# Helper function to draw network
def draw_network(ax, network, path=None, explored=None, title=""):
    # Draw all edges
    for node, neighbors in network.graph.items():
        x1, y1 = network.positions[node]
        for neighbor, weight in neighbors:
            if node < neighbor:  # Avoid duplicate edges
                x2, y2 = network.positions[neighbor]
                ax.plot([x1, x2], [y1, y2], 'k-', linewidth=1, alpha=0.3, zorder=1)
                # Edge label (weight)
                mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
                ax.text(mid_x, mid_y, f'{weight}', fontsize=8, 
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
    
    # Draw explored nodes
    if explored:
        explored_positions = [network.positions[n] for n in explored if n in network.positions]
        if explored_positions:
            ex_x, ex_y = zip(*explored_positions)
            ax.scatter(ex_x, ex_y, c='lightblue', s=400, zorder=2, alpha=0.6, label='Explored')
    
    # Draw all nodes
    for node, (x, y) in network.positions.items():
        color = 'lightgray'
        if node == start:
            color = 'green'
        elif node == goal:
            color = 'red'
        ax.scatter([x], [y], c=color, s=500, zorder=3, edgecolors='black', linewidths=2)
        ax.text(x, y, node, fontsize=12, ha='center', va='center', fontweight='bold', zorder=4)
    
    # Draw path
    if path:
        path_positions = [network.positions[n] for n in path]
        px, py = zip(*path_positions)
        ax.plot(px, py, 'b-', linewidth=4, alpha=0.7, zorder=5, label='Path')
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    ax.set_aspect('equal')

# Plot 1: Dijkstra
ax1 = fig.add_subplot(2, 3, 1)
draw_network(ax1, network, dijkstra_path, dijkstra_explored, 
             f"Dijkstra's Algorithm\nPath: {' → '.join(dijkstra_path)}\nCost: {dijkstra_cost:.1f}")

# Plot 2: A* (Euclidean)
ax2 = fig.add_subplot(2, 3, 2)
draw_network(ax2, network, astar_path, astar_explored, 
             f"A* (Euclidean Heuristic)\nPath: {' → '.join(astar_path)}\nCost: {astar_cost:.1f}")

# Plot 3: A* (Manhattan)
ax3 = fig.add_subplot(2, 3, 3)
draw_network(ax3, network, astar_manhattan_path, astar_manhattan_explored, 
             f"A* (Manhattan Heuristic)\nPath: {' → '.join(astar_manhattan_path)}\nCost: {astar_manhattan_cost:.1f}")

# Plot 4: Weighted A* (ε=1.5)
ax4 = fig.add_subplot(2, 3, 4)
draw_network(ax4, network, weighted_astar_path, weighted_astar_explored, 
             f"Weighted A* (ε=1.5)\nPath: {' → '.join(weighted_astar_path)}\nCost: {weighted_astar_cost:.1f}")

# Plot 5: Comparison table
ax5 = fig.add_subplot(2, 3, 5)
ax5.axis('off')

comparison_data = [
    ['Algorithm', 'Path Cost', 'Nodes Explored', 'Path Length'],
    ['Dijkstra', f'{dijkstra_cost:.1f}', len(dijkstra_explored), len(dijkstra_path)],
    ['A* (Euclidean)', f'{astar_cost:.1f}', len(astar_explored), len(astar_path)],
    ['A* (Manhattan)', f'{astar_manhattan_cost:.1f}', len(astar_manhattan_explored), len(astar_manhattan_path)],
    ['Weighted A* (ε=1.5)', f'{weighted_astar_cost:.1f}', len(weighted_astar_explored), len(weighted_astar_path)],
]

table = ax5.table(cellText=comparison_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Header row formatting
for i in range(4):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax5.set_title('Algorithm Comparison', fontsize=14, fontweight='bold', pad=20)

# Plot 6: Explored nodes bar chart
ax6 = fig.add_subplot(2, 3, 6)

algorithms = ['Dijkstra', 'A*\n(Euclidean)', 'A*\n(Manhattan)', 'Weighted A*\n(ε=1.5)']
explored_counts = [
    len(dijkstra_explored),
    len(astar_explored),
    len(astar_manhattan_explored),
    len(weighted_astar_explored)
]
colors = ['gray', 'blue', 'green', 'orange']

bars = ax6.bar(algorithms, explored_counts, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, count in zip(bars, explored_counts):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height,
             f'{count}',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

ax6.set_ylabel('Nodes Explored', fontsize=12)
ax6.set_title('Search Efficiency Comparison', fontsize=14, fontweight='bold')
ax6.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed results
print("=" * 80)
print("ROUTE PLANNING COMPARISON")
print("=" * 80)
print(f"Start: {start}, Goal: {goal}")
print(f"Total nodes in network: {len(network.graph)}")
print("\n" + "-" * 80)
print(f"{'Algorithm':<25} {'Path Cost':<12} {'Explored':<12} {'Path'}")
print("-" * 80)
print(f"{'Dijkstra':<25} {dijkstra_cost:<12.1f} {len(dijkstra_explored):<12} {' → '.join(dijkstra_path)}")
print(f"{'A* (Euclidean)':<25} {astar_cost:<12.1f} {len(astar_explored):<12} {' → '.join(astar_path)}")
print(f"{'A* (Manhattan)':<25} {astar_manhattan_cost:<12.1f} {len(astar_manhattan_explored):<12} {' → '.join(astar_manhattan_path)}")
print(f"{'Weighted A* (ε=1.5)':<25} {weighted_astar_cost:<12.1f} {len(weighted_astar_explored):<12} {' → '.join(weighted_astar_path)}")
print("-" * 80)

# Efficiency analysis
dijkstra_efficiency = (len(network.graph) - len(dijkstra_explored)) / len(network.graph) * 100
astar_efficiency = (len(network.graph) - len(astar_explored)) / len(network.graph) * 100

print(f"\nEfficiency Analysis:")
print(f"  Dijkstra explored {len(dijkstra_explored)}/{len(network.graph)} nodes ({100-dijkstra_efficiency:.1f}%)")
print(f"  A* explored {len(astar_explored)}/{len(network.graph)} nodes ({100-astar_efficiency:.1f}%)")
print(f"  Improvement: {(1 - len(astar_explored)/len(dijkstra_explored))*100:.1f}% fewer nodes")
print("=" * 80)

### Dijkstra's Algorithm

**Dijkstra's algorithm** finds the shortest path from a start node to all other nodes in a weighted graph.

**Algorithm**:
1. Initialize distances: $d[s] = 0$, $d[v] = \infty$ for all $v \neq s$
2. Create priority queue $Q$ with all nodes
3. While $Q$ is not empty:
   - Extract node $u$ with minimum $d[u]$
   - For each neighbor $v$ of $u$:
     - If $d[u] + w(u, v) < d[v]$:
       - $d[v] = d[u] + w(u, v)$ (relaxation)
       - Update predecessor: $\pi[v] = u$

**Properties**:
- **Completeness**: Always finds a solution if one exists
- **Optimality**: Finds the shortest path
- **Time Complexity**: $O((|V| + |E|) \log |V|)$ with binary heap
- **Space Complexity**: $O(|V|)$

**Limitations**:
- Explores uniformly in all directions (no goal bias)
- Can be slow on large graphs

---

### A* Search

**A*** improves upon Dijkstra by using a **heuristic function** $h(n)$ to guide search towards the goal.

**Cost Function**:
$$f(n) = g(n) + h(n)$$

Where:
- $g(n)$: Cost from start to node $n$ (like Dijkstra)
- $h(n)$: **Heuristic** - estimated cost from $n$ to goal
- $f(n)$: Estimated total cost through $n$

**Algorithm**: Same as Dijkstra, but prioritize by $f(n)$ instead of $g(n)$ alone.

**Heuristic Requirements**:

**Admissible**: $h(n) \leq h^*(n)$ (never overestimate)
- Ensures optimality

**Consistent** (Monotone): $h(n) \leq c(n, n') + h(n')$ for all neighbors $n'$
- Ensures A* expands each node at most once
- Triangle inequality

**Common Heuristics for 2D Navigation**:

1. **Euclidean Distance** (straight-line):
   $$h(n) = \sqrt{(x_n - x_g)^2 + (y_n - y_g)^2}$$
   - Admissible for any path
   - Best for open spaces

2. **Manhattan Distance** (grid-aligned):
   $$h(n) = |x_n - x_g| + |y_n - y_g|$$
   - Admissible for grid graphs
   - Best when only 4-directional movement

3. **Octile Distance** (diagonal allowed):
   $$h(n) = \max(|x_n - x_g|, |y_n - y_g|) + (\sqrt{2} - 1) \min(|x_n - x_g|, |y_n - y_g|)$$
   - Accounts for diagonal moves

**Heuristic Weight**:
- $h(n) = 0$: A* becomes Dijkstra (guaranteed optimal, slow)
- $h(n)$ admissible: A* is optimal
- $h(n) > h^*(n)$ (inadmissible): Faster but may not find optimal path

**Weighted A***: $f(n) = g(n) + \epsilon \cdot h(n)$ with $\epsilon > 1$
- Trades optimality for speed
- $\epsilon = 1.5$ is common (50% faster, ~10% longer paths)

---

### Comparison: Dijkstra vs A*

| Aspect | Dijkstra | A* |
|--------|----------|-----|
| **Heuristic** | No ($h(n) = 0$) | Yes |
| **Nodes Explored** | Many (uniform expansion) | Few (goal-directed) |
| **Optimality** | Always optimal | Optimal if $h$ admissible |
| **Speed** | Slower | Faster (with good heuristic) |
| **Use Case** | All-pairs shortest paths | Single goal |

**When to use**:
- **Dijkstra**: Need paths to multiple destinations
- **A***: Single goal, good heuristic available

In [ ]:
# Finite State Machine for Lane Change Behavior

class DrivingState(Enum):
    """Enum for driving states"""
    LANE_KEEPING = "Lane Keeping"
    PREP_LANE_CHANGE_LEFT = "Prepare Lane Change Left"
    LANE_CHANGE_LEFT = "Executing Lane Change Left"
    PREP_LANE_CHANGE_RIGHT = "Prepare Lane Change Right"
    LANE_CHANGE_RIGHT = "Executing Lane Change Right"
    EMERGENCY_STOP = "Emergency Stop"


class HighwayBehaviorFSM:
    """Finite State Machine for highway driving behavior"""
    
    def __init__(self):
        self.state = DrivingState.LANE_KEEPING
        self.lane = 1  # Current lane (0=left, 1=center, 2=right on 3-lane highway)
        self.state_history = [self.state]
        self.lane_change_timer = 0
        self.lane_change_duration = 30  # steps to complete lane change
        
    def update(self, ego_speed, front_vehicle_speed, front_vehicle_dist,
               left_lane_clear, right_lane_clear, emergency):
        """
        Update FSM based on current conditions
        
        Parameters:
        - ego_speed: Speed of our vehicle
        - front_vehicle_speed: Speed of vehicle ahead
        - front_vehicle_dist: Distance to vehicle ahead
        - left_lane_clear: Is left lane safe for lane change?
        - right_lane_clear: Is right lane safe for lane change?
        - emergency: Emergency condition detected?
        """
        old_state = self.state
        
        # Emergency override
        if emergency:
            self.state = DrivingState.EMERGENCY_STOP
        
        # State machine logic
        elif self.state == DrivingState.LANE_KEEPING:
            # Check if vehicle ahead is slow
            if front_vehicle_dist < 50 and front_vehicle_speed < ego_speed - 5:
                # Try to change lanes
                if self.lane > 0 and left_lane_clear:
                    self.state = DrivingState.PREP_LANE_CHANGE_LEFT
                elif self.lane < 2 and right_lane_clear:
                    self.state = DrivingState.PREP_LANE_CHANGE_RIGHT
        
        elif self.state == DrivingState.PREP_LANE_CHANGE_LEFT:
            if left_lane_clear:
                # Safe to change
                self.state = DrivingState.LANE_CHANGE_LEFT
                self.lane_change_timer = 0
            else:
                # Not safe anymore, abort
                self.state = DrivingState.LANE_KEEPING
        
        elif self.state == DrivingState.LANE_CHANGE_LEFT:
            self.lane_change_timer += 1
            if self.lane_change_timer >= self.lane_change_duration:
                # Lane change complete
                self.lane -= 1
                self.state = DrivingState.LANE_KEEPING
                self.lane_change_timer = 0
        
        elif self.state == DrivingState.PREP_LANE_CHANGE_RIGHT:
            if right_lane_clear:
                self.state = DrivingState.LANE_CHANGE_RIGHT
                self.lane_change_timer = 0
            else:
                self.state = DrivingState.LANE_KEEPING
        
        elif self.state == DrivingState.LANE_CHANGE_RIGHT:
            self.lane_change_timer += 1
            if self.lane_change_timer >= self.lane_change_duration:
                self.lane += 1
                self.state = DrivingState.LANE_KEEPING
                self.lane_change_timer = 0
        
        elif self.state == DrivingState.EMERGENCY_STOP:
            # Stay in emergency stop until resolved
            if not emergency:
                self.state = DrivingState.LANE_KEEPING
        
        # Record state transition
        if old_state != self.state:
            self.state_history.append(self.state)
            print(f"State transition: {old_state.value} → {self.state.value}")
        
        return self.get_action()
    
    def get_action(self):
        """Return action based on current state"""
        if self.state == DrivingState.LANE_KEEPING:
            return "maintain_speed"
        elif self.state == DrivingState.PREP_LANE_CHANGE_LEFT:
            return "signal_left"
        elif self.state == DrivingState.LANE_CHANGE_LEFT:
            return "steer_left"
        elif self.state == DrivingState.PREP_LANE_CHANGE_RIGHT:
            return "signal_right"
        elif self.state == DrivingState.LANE_CHANGE_RIGHT:
            return "steer_right"
        elif self.state == DrivingState.EMERGENCY_STOP:
            return "brake_hard"
        return "maintain_speed"


# Simulate highway driving scenario
def simulate_highway_driving():
    fsm = HighwayBehaviorFSM()
    
    # Simulation parameters
    num_steps = 200
    ego_speed = 30  # m/s
    
    # Data recording
    states_over_time = []
    lanes_over_time = []
    actions_over_time = []
    
    print("Starting highway driving simulation...\n")
    
    for t in range(num_steps):
        # Simulate environment (simple scenario)
        if t < 50:
            # No obstacles
            front_vehicle_speed = ego_speed
            front_vehicle_dist = 100
            left_lane_clear = True
            right_lane_clear = True
            emergency = False
        elif t < 100:
            # Slow vehicle ahead
            front_vehicle_speed = 20
            front_vehicle_dist = 40
            left_lane_clear = True
            right_lane_clear = False
            emergency = False
        elif t < 150:
            # Back in left lane, slow vehicle in left lane
            front_vehicle_speed = 25
            front_vehicle_dist = 45
            left_lane_clear = False
            right_lane_clear = True
            emergency = False
        else:
            # Clear road
            front_vehicle_speed = ego_speed
            front_vehicle_dist = 100
            left_lane_clear = True
            right_lane_clear = True
            emergency = False
        
        # Update FSM
        action = fsm.update(ego_speed, front_vehicle_speed, front_vehicle_dist,
                           left_lane_clear, right_lane_clear, emergency)
        
        # Record
        states_over_time.append(fsm.state)
        lanes_over_time.append(fsm.lane)
        actions_over_time.append(action)
    
    # Visualization
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # State over time
    ax1 = axes[0]
    state_values = [list(DrivingState).index(s) for s in states_over_time]
    state_names = [s.value for s in DrivingState]
    
    ax1.plot(state_values, 'b-', linewidth=2)
    ax1.set_yticks(range(len(DrivingState)))
    ax1.set_yticklabels(state_names, fontsize=9)
    ax1.set_ylabel('Driving State')
    ax1.set_title('FSM State Over Time')
    ax1.grid(True, alpha=0.3)
    
    # Lane over time
    ax2 = axes[1]
    ax2.plot(lanes_over_time, 'g-', linewidth=2)
    ax2.set_yticks([0, 1, 2])
    ax2.set_yticklabels(['Left Lane', 'Center Lane', 'Right Lane'])
    ax2.set_ylabel('Current Lane')
    ax2.set_xlabel('Time Step')
    ax2.set_title('Lane Position Over Time')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-0.5, 2.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSimulation complete!")
    print(f"State transitions: {len(fsm.state_history) - 1}")
    print(f"Final lane: {['Left', 'Center', 'Right'][fsm.lane]}")

simulate_highway_driving()

## Exercises

### Exercise 1: Bidirectional A* Search

Implement bidirectional A* which searches from both start and goal simultaneously, meeting in the middle.

**Tasks:**
1. Implement bidirectional A* search
2. Compare with standard A* in terms of nodes explored
3. Test on larger road networks
4. Analyze when bidirectional search helps most

**Hint:** Run two A* searches simultaneously and stop when they meet.

In [ ]:
# Exercise 3 - Template

def time_dependent_weight(edge, current_time):
    """
    TODO: Return edge weight based on time of day
    
    Example: Highway edges are slower during rush hour
    """
    pass

def time_dependent_astar(network, start, goal, departure_time):
    """
    TODO: Implement A* with time-dependent weights
    
    Track current time as search progresses
    """
    pass

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Artificial Intelligence: A Modern Approach** by Russell & Norvig (4th Edition, 2020)
   - Chapter 3: Solving Problems by Searching
   - Dijkstra's, A*, and heuristic search

2. **Planning Algorithms** by LaValle (2006)
   - Comprehensive coverage of motion planning
   - Available free online: http://planning.cs.uiuc.edu/

3. **Automated Planning and Acting** by Ghallab, Nau, & Traverso (2016)
   - FSM, hierarchical task networks, decision-theoretic planning

### Key Papers

1. **A* Search**
   - Hart, Nilsson, & Raphael (1968) - "A Formal Basis for the Heuristic Determination of Minimum Cost Paths"
   - Original A* paper

2. **Behavior Planning**
   - Ulbrich & Maurer (2013) - "Probabilistic Online POMDP Decision Making for Lane Changes"
   - Wei et al. (2013) - "Towards a Viable Autonomous Driving Research Platform"

3. **Hierarchical Planning**
   - Kaelbling & Lozano-Pérez (2013) - "Integrated Task and Motion Planning in Belief Space"

### Algorithms & Data Structures

1. **Graph Algorithms**
   - Priority queue (heap) for Dijkstra/A*
   - Bidirectional search
   - D* Lite for dynamic replanning

2. **Behavior Planning Approaches**
   - Finite State Machines (FSM)
   - Behavior trees
   - Markov Decision Processes (MDP)
   - Partially Observable MDPs (POMDP)

### Software & Tools

1. **Route Planning Libraries**
   - **NetworkX** (Python): Graph algorithms - https://networkx.org/
   - **OSRM**: Open Source Routing Machine - http://project-osrm.org/
   - **Graphhopper**: Java routing engine - https://www.graphhopper.com/

2. **Behavior Planning**
   - **py_trees**: Behavior trees in Python - https://github.com/splintered-reality/py_trees
   - **SMACH**: ROS state machine library

3. **Simulation**
   - **CARLA**: Open-source autonomous driving simulator
   - **SUMO**: Traffic simulation

### Online Resources

1. **Coursera: Motion Planning for Self-Driving Cars**
   - University of Toronto
   - Covers A*, lattice planning, behavior planning

2. **MIT OpenCourseWare: Artificial Intelligence**
   - Search algorithms and planning

3. **Red Blob Games: A* Tutorial**
   - https://www.redblobgames.com/pathfinding/a-star/
   - Interactive visualizations

### Applications in Autonomous Vehicles

1. **Mission Planning Hierarchy**
   - **Mission**: Route from A to B (Dijkstra/A*)
   - **Behavior**: Lane changes, merging, intersection handling (FSM)
   - **Motion**: Trajectory generation (next week's topic)

2. **Real-World Considerations**
   - Dynamic replanning (road closures, accidents)
   - Multi-objective optimization (time, safety, comfort, fuel)
   - Uncertainty handling
   - Real-time constraints

3. **Industry Practices**
   - Waymo: Hierarchical planning with prediction
   - Tesla: Neural network-based planning
   - Cruise: Rule-based + learning hybrid

### Advanced Topics

1. **Learning-Based Planning**
   - Imitation learning for behavior cloning
   - Reinforcement learning for policy optimization
   - Inverse reinforcement learning for cost function learning

2. **Probabilistic Planning**
   - Monte Carlo Tree Search (MCTS)
   - POMDP solvers
   - Contingency planning

3. **Multi-Agent Planning**
   - Game theory for interaction
   - Joint planning with other vehicles
   - Negotiation and communication

---

## Summary

This notebook covered mission and behavior planning for autonomous vehicles:

**Key Concepts:**
- Route planning on road networks (graphs)
- Dijkstra's algorithm for shortest paths
- A* search with heuristics for efficient planning
- Finite State Machines for behavior decision-making

**Practical Skills:**
- Implementing graph-based search algorithms
- Designing FSMs for driving behaviors
- Comparing algorithm efficiency
- Handling state transitions

**Next Steps:**
1. Study trajectory generation and motion planning
2. Explore learning-based behavior planning
3. Implement behavior trees for complex scenarios
4. Learn about POMDP for uncertainty handling